In [1]:
!pip install ktrain
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_excel('/content/8emotionscombineddata .xlsx')

In [3]:
test_data = pd.read_excel('/content/testdata.xlsx')

In [4]:
train_text_column = 'text'
train_label_column = 'emot2'

In [5]:
test_text_column = 'text'

In [6]:
unique_emotions = train_data[train_label_column].unique()
print(unique_emotions)

['affinity' 'compassion' 'dismay' 'fear' 'confidence' 'careless' 'anger']


In [7]:
X_train = train_data[train_text_column]
y_train = train_data[train_label_column]
print(X_train)
print(y_train)

0       I want to protect children and this bill simpl...
1       I want to protect children and this bill simpl...
2       Gender Dysphoria is a real and challenging med...
3       Those who experience it deserve our compassion...
4       The recent increase in those claiming to be tr...
                              ...                        
1599    Vajani Telos, the co-owner of Greek Bites food...
1600    This is a time when we raise the\nvoice of agr...
1601    Weld County and the Colorado Cattlemen's Assoc...
1602    Still, Polis' proclamation caught the ire of m...
1603    'But still, you matter, your voice matters, an...
Name: text, Length: 1604, dtype: object
0         affinity
1       compassion
2           dismay
3       compassion
4           dismay
           ...    
1599        dismay
1600      affinity
1601        dismay
1602         anger
1603    compassion
Name: emot2, Length: 1604, dtype: object


In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print(X_train)

705     I encourage him to come out to\nthe Eastern Pl...
962     He said the Colorado proclamation 'is a direct...
599     Well, I've heard from a lot of doctors, and we...
411     And for as far back as I can remember, this ca...
710     'For our governor to say that we should have a...
                              ...                        
1130    ograms. These taxes disproportionately impact ...
1294    he Colorado Dairy Farmers, Colorado Egg Produc...
860                                We need to fight back.
1459    The insult felt by ranchers and others in the ...
1126    “In Oklahoma, we’re always going to fight for ...
Name: text, Length: 1283, dtype: object


In [9]:
X_train = X_train.values.tolist()
y_train = y_train.values.tolist()
X_val = X_val.values.tolist()
y_val = y_val.values.tolist()

In [10]:
(X_train, y_train), (X_val, y_val), preproc = text.texts_from_array(x_train=X_train,
                                                                   y_train=y_train,
                                                                   x_test=X_val,
                                                                   y_test=y_val,
                                                                   class_names=unique_emotions,
                                                                   preprocess_mode='bert',
                                                                   maxlen=500)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


/usr/local/lib/python3.10/dist-packages/ktrain/utils.py:744: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  warnings.warn(


task: text classification


In [11]:
model = text.text_classifier(name='bert',
                             train_data=(X_train, y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 500


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


done.


In [12]:
learner = ktrain.get_learner(model=model,
                              train_data=(X_train, y_train),
                              val_data=(X_val, y_val),
                              batch_size=6)

In [13]:
learner.fit_onecycle(lr=2e-5, epochs=100)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/100
214/214 [==============================] - 202s 850ms/step - loss: 1.7998 - accuracy: 0.3414 - val_loss: 1.6322 - val_accuracy: 0.4330
Epoch 2/100
214/214 [==============================] - 182s 851ms/step - loss: 1.5750 - accuracy: 0.4513 - val_loss: 1.5201 - val_accuracy: 0.4766
Epoch 3/100
214/214 [==============================] - 182s 851ms/step - loss: 1.4231 - accuracy: 0.5456 - val_loss: 1.4667 - val_accuracy: 0.5109
Epoch 4/100
214/214 [==============================] - 182s 851ms/step - loss: 1.2958 - accuracy: 0.5916 - val_loss: 1.4139 - val_accuracy: 0.5171
Epoch 5/100
214/214 [==============================] - 182s 851ms/step - loss: 1.1613 - accuracy: 0.6196 - val_loss: 1.4000 - val_accuracy: 0.5234
Epoch 6/100
214/214 [==============================] - 182s 851ms/step - loss: 1.0501 - accuracy: 0.6555 - val_loss: 1.3761 - val_accuracy: 0.5234
Epoch 7/100
214/214 [==============================] - 

In [14]:
X_test = test_data[test_text_column]

In [15]:
X_test = preproc.preprocess_test(X_test)

preprocessing test...
language: en


In [16]:
predicted_probabilities = learner.predict(X_test)
predicted_emotions = [unique_emotions[label] for label in predicted_probabilities.argmax(axis=1)]

9/9 [==============================] - 14s 1s/step


In [17]:
test_label_column = 'emot2'

In [18]:
for text, actual_emotion, predicted_emotion in zip(test_data[test_text_column], test_data[test_label_column], predicted_emotions):
    print(f"Text: {text}")
    print(f"Actual Emotion: {actual_emotion}")
    print(f"Predicted Emotion: {predicted_emotion}")
    print("-" * 50)

Text: I'm pleased to be here in support of Senate Bill 5313.
Actual Emotion: affinity
Predicted Emotion: affinity
--------------------------------------------------
Text: We have done so much over the last couple of years to expand access to health care and was such a thrill to see you here cascade care 2.0
Actual Emotion: affinity
Predicted Emotion: affinity
--------------------------------------------------
Text: I really salute Senator frock for his leadership and getting more coverage to more Washingtonians
Actual Emotion: affinity
Predicted Emotion: affinity
--------------------------------------------------
Text: We've also done more to tear down discrimination against members of the LGBTQ community and accessing health care.
Actual Emotion: dismay
Predicted Emotion: fear
--------------------------------------------------
Text: Appreciate the passage of the Reproductive Health Access for All Act as an example of that
Actual Emotion: affinity
Predicted Emotion: affinity
----------